In [ ]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.0-bin-hadoop3.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# After downloading dataset, you should have a file "network.csv" in your Files (click the folder icon in the left sidebar)
!wget https://bhooi.github.io/teaching/cs4225/bomb/part2/network.csv

--2022-10-15 04:07:43--  https://bhooi.github.io/teaching/cs4225/bomb/part2/network.csv
Resolving bhooi.github.io (bhooi.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to bhooi.github.io (bhooi.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 897278 (876K) [text/csv]
Saving to: ‘network.csv.1’

network.csv.1       100%[===================>] 876.25K  --.-KB/s    in 0.05s   

2022-10-15 04:07:43 (18.7 MB/s) - ‘network.csv.1’ saved [897278/897278]



In [ ]:
df = spark.read.option('header', True).csv("network.csv").repartition(1).cache()
df.toPandas()

,src,dst,day
0,172.016.117.052,207.025.071.144,0
1,230.001.010.020,172.016.112.050,0
2,209.074.060.168,172.016.113.050,0
3,010.020.030.040,172.016.112.050,0
4,230.001.010.020,172.016.112.050,0
...,...,...,...
25877,208.239.005.230,192.168.001.001,29
25878,199.174.194.123,172.016.114.050,29
25879,166.102.114.043,172.016.113.050,29
25880,172.016.116.201,207.046.130.168,29


In [ ]:
unique_dst_visits = df.select(['src', 'dst']).dropDuplicates().groupBy('dst').count()
thirteen_unique_visitors = unique_dst_visits.filter(unique_dst_visits['count'] == 13)
num_weeks_visited = df.filter(df.day % 7 == 0).join(thirteen_unique_visitors, ['dst']).groupBy(['dst', 'day']).count().groupBy('dst').count()
num_weeks_visited.filter(num_weeks_visited['count'] > 4).toPandas()

,dst,count
0,207.025.071.025,5
